<a href="https://colab.research.google.com/github/leonardo3108/IA368dd/blob/main/main_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download the CISI dataset

> Before you begin, make sure you have downloaded the CISI dataset and stored it in a directory on your local machine. You can download the dataset from this link: https://ir.shef.ac.uk/cloughie/resources/cisi/cisi.tar.gz


O caminho fornecido pelo ChatGPT estava incorreto, modifiquei para https://ir.dcs.gla.ac.uk/resources/test_collections/cisi/cisi.tar.gz


In [3]:
!wget https://ir.dcs.gla.ac.uk/resources/test_collections/cisi/cisi.tar.gz

--2023-02-18 19:18:54--  https://ir.dcs.gla.ac.uk/resources/test_collections/cisi/cisi.tar.gz
Resolving ir.dcs.gla.ac.uk (ir.dcs.gla.ac.uk)... 130.209.240.253
Connecting to ir.dcs.gla.ac.uk (ir.dcs.gla.ac.uk)|130.209.240.253|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 775144 (757K) [application/gzip]
Saving to: ‘cisi.tar.gz’

cisi.tar.gz         100%[===================>] 756.98K  1.39MB/s    in 0.5s    

2023-02-18 19:18:55 (1.39 MB/s) - ‘cisi.tar.gz’ saved [775144/775144]



In [5]:
!tar -xzvf cisi.tar.gz

CISI.ALL
CISI.BLN
CISI.QRY
CISI.REL


# Install and import needed packages

In [11]:
!pip install nltk
!pip install rank-bm25

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


> from bm25 import BM25Okapi

The package name ChatGPT gave was incorrect. I've got the correct from https://pypi.org/project/rank-bm25/

In [12]:
import os
import re
import json
import nltk
from rank_bm25 import BM25Okapi

# Set up the NLTK stopwords

I had to add to ChatGPT generated code the 'punkt' package from NLTK for correct tokenization.

In [24]:
nltk.download('stopwords')
nltk.download('punkt')
stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(['from', 'subject', 're', 'edu', 'use'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Define a function to preprocess the documents and queries

Into the ChatGPT code, I had to add a treatment for line breaks. It was adding words from different lines together.

In [73]:
def preprocess(text):
    # Convert the text to lowercase
    text = text.lower()

    # Substitute line breaks for space
    text = re.sub(r'\n', ' ', text)

    # Remove non-alphanumeric characters
    text = re.sub(r'[^a-z0-9 ]', '', text)

    # Tokenize the text
    tokens = nltk.word_tokenize(text)

    # Remove stopwords
    tokens = [t for t in tokens if t not in stopwords]

    # Stem the tokens
    stemmer = nltk.PorterStemmer()
    tokens = [stemmer.stem(t) for t in tokens]

    return tokens


In [74]:
tokenized_query = preprocess('''What problems and concerns are there in making up descriptive titles?
What difficulties are involved in automatically retrieving articles from
approximate titles?''')
print(tokenized_query)

['problem', 'concern', 'make', 'descript', 'titl', 'difficulti', 'involv', 'automat', 'retriev', 'articl', 'approxim', 'titl']


# Load the documents

The ChatGPT code was interpretating badly the CISI file format. The title (.T) and author (.A) sessions were ignorated, and cross references(.X) session should be ignored. 

Got help from https://www.kaggle.com/code/kerneler/starter-cisi-c3822057-1/data.

I adjusted the code.

In [76]:
docs_path = 'CISI.ALL'
docs = []
with open(docs_path, 'r') as f:
    doc_text = ''
    register = False
    for line in f:
        if line.startswith('.I'):
            if doc_text:
                docs.append(doc_text)
            doc_text = ''
            register = True
        elif line.startswith('.X'):
            register = False
        elif register and not line.startswith('.'):
            doc_text += line
    if doc_text:
        docs.append(doc_text)

In [77]:
docs[0]

"18 Editions of the Dewey Decimal Classifications\nComaromi, J.P.\n   The present study is a history of the DEWEY Decimal\nClassification.  The first edition of the DDC was published\nin 1876, the eighteenth edition in 1971, and future editions\nwill continue to appear as needed.  In spite of the DDC's\nlong and healthy life, however, its full story has never\nbeen told.  There have been biographies of Dewey\nthat briefly describe his system, but this is the first\nattempt to provide a detailed history of the work that\nmore than any other has spurred the growth of\nlibrarianship in this country and abroad.\n"

In [78]:
preprocess(docs[0])

['18',
 'edit',
 'dewey',
 'decim',
 'classif',
 'comaromi',
 'jp',
 'present',
 'studi',
 'histori',
 'dewey',
 'decim',
 'classif',
 'first',
 'edit',
 'ddc',
 'publish',
 '1876',
 'eighteenth',
 'edit',
 '1971',
 'futur',
 'edit',
 'continu',
 'appear',
 'need',
 'spite',
 'ddc',
 'long',
 'healthi',
 'life',
 'howev',
 'full',
 'stori',
 'never',
 'told',
 'biographi',
 'dewey',
 'briefli',
 'describ',
 'system',
 'first',
 'attempt',
 'provid',
 'detail',
 'histori',
 'work',
 'spur',
 'growth',
 'librarianship',
 'countri',
 'abroad']

# Preprocess the documents

In [79]:
docs = [preprocess(doc) for doc in docs]

In [80]:
len(docs), docs[:5]

(1460,
 [['18',
   'edit',
   'dewey',
   'decim',
   'classif',
   'comaromi',
   'jp',
   'present',
   'studi',
   'histori',
   'dewey',
   'decim',
   'classif',
   'first',
   'edit',
   'ddc',
   'publish',
   '1876',
   'eighteenth',
   'edit',
   '1971',
   'futur',
   'edit',
   'continu',
   'appear',
   'need',
   'spite',
   'ddc',
   'long',
   'healthi',
   'life',
   'howev',
   'full',
   'stori',
   'never',
   'told',
   'biographi',
   'dewey',
   'briefli',
   'describ',
   'system',
   'first',
   'attempt',
   'provid',
   'detail',
   'histori',
   'work',
   'spur',
   'growth',
   'librarianship',
   'countri',
   'abroad'],
  ['made',
   'technic',
   'librari',
   'slater',
   'report',
   'analysi',
   '6300',
   'act',
   '104',
   'technic',
   'librari',
   'unit',
   'kingdom',
   'librari',
   'one',
   'aspect',
   'wider',
   'pattern',
   'inform',
   'inform',
   'transfer',
   'librari',
   'restrict',
   'document',
   'take',
   'account',
   'd

# Build the BM25 index

In [81]:
bm25 = BM25Okapi(docs)

In [82]:
bm25.get_scores(tokenized_query)

array([0.        , 0.        , 0.        , ..., 1.95827862, 1.77080883,
       0.        ])

# Load the queries

In [83]:
queries_path = 'CISI.QRY'
queries = []
with open(queries_path, 'r') as f:
    query_text = ''
    for line in f:
        if line.startswith('.I'):
            if query_text:
                queries.append(query_text)
            query_text = ''
        elif line.startswith('.W'):
            query_text = line[3:]
        else:
            query_text += line
    if query_text:
        queries.append(query_text)

In [84]:
queries[0]

'What problems and concerns are there in making up descriptive titles?\nWhat difficulties are involved in automatically retrieving articles from\napproximate titles?\nWhat is the usual relevance of the content of articles to their titles?\n'

# Preprocess the queries

In [85]:
queries = [preprocess(query) for query in queries]

In [86]:
queries[0]

['problem',
 'concern',
 'make',
 'descript',
 'titl',
 'difficulti',
 'involv',
 'automat',
 'retriev',
 'articl',
 'approxim',
 'titl',
 'usual',
 'relev',
 'content',
 'articl',
 'titl']

# Perform the queries

ChatGPT code was ok here. Just checked.

In [88]:
# Perform the queries
results = {}
for i, query in enumerate(queries):
    doc_scores = bm25.get_scores(query)
    top_docs = sorted(range(len(doc_scores)), key=lambda i: doc_scores[i], reverse=True)[:100]
    results[i+1] = top_docs

In [98]:
doc_scores

array([ 7.44076952,  9.63984534,  0.3359594 , ...,  1.17052702,
        6.7964225 , 10.84566393])

In [101]:
sorted(doc_scores, reverse = True)[:10]

[41.853778859208425,
 39.52711178679075,
 34.53808349251481,
 33.42141397628546,
 31.50818598575653,
 31.47480646879566,
 31.37205446328315,
 31.182037695591223,
 31.09846049855547,
 30.929022306467875]

In [102]:
results[1]

[428,
 721,
 1298,
 758,
 64,
 75,
 37,
 1420,
 710,
 602,
 819,
 927,
 1280,
 1089,
 665,
 1054,
 588,
 1090,
 509,
 1194,
 812,
 540,
 1008,
 999,
 41,
 649,
 482,
 63,
 619,
 1264,
 789,
 905,
 465,
 581,
 1123,
 798,
 1322,
 85,
 321,
 211,
 850,
 1418,
 134,
 446,
 275,
 488,
 787,
 746,
 1363,
 218,
 1223,
 675,
 76,
 194,
 782,
 957,
 847,
 1163,
 1117,
 766,
 1161,
 829,
 868,
 952,
 149,
 51,
 1293,
 200,
 874,
 802,
 268,
 401,
 70,
 414,
 679,
 494,
 523,
 1063,
 920,
 195,
 991,
 478,
 224,
 1001,
 893,
 654,
 977,
 191,
 885,
 559,
 1029,
 1088,
 419,
 725,
 412,
 314,
 188,
 1435,
 768,
 714]

# Write the results to a file

In [103]:
with open('bm25_results.json', 'w') as f:
    json.dump(results, f)

# Print the top 10 results for each query

In [104]:
for query_id, doc_ids in results.items():
    print('Query', query_id)
    for i, doc_id in enumerate(doc_ids[:10]):
        print('\tRank {}: Doc {}'.format(i+1, doc_id))

Query 1
	Rank 1: Doc 428
	Rank 2: Doc 721
	Rank 3: Doc 1298
	Rank 4: Doc 758
	Rank 5: Doc 64
	Rank 6: Doc 75
	Rank 7: Doc 37
	Rank 8: Doc 1420
	Rank 9: Doc 710
	Rank 10: Doc 602
Query 2
	Rank 1: Doc 1137
	Rank 2: Doc 308
	Rank 3: Doc 596
	Rank 4: Doc 796
	Rank 5: Doc 789
	Rank 6: Doc 487
	Rank 7: Doc 419
	Rank 8: Doc 1070
	Rank 9: Doc 450
	Rank 10: Doc 1326
Query 3
	Rank 1: Doc 1180
	Rank 2: Doc 539
	Rank 3: Doc 468
	Rank 4: Doc 1178
	Rank 5: Doc 1132
	Rank 6: Doc 1076
	Rank 7: Doc 1234
	Rank 8: Doc 1276
	Rank 9: Doc 913
	Rank 10: Doc 1141
Query 4
	Rank 1: Doc 600
	Rank 2: Doc 419
	Rank 3: Doc 76
	Rank 4: Doc 745
	Rank 5: Doc 178
	Rank 6: Doc 564
	Rank 7: Doc 174
	Rank 8: Doc 789
	Rank 9: Doc 71
	Rank 10: Doc 1293
Query 5
	Rank 1: Doc 559
	Rank 2: Doc 1104
	Rank 3: Doc 482
	Rank 4: Doc 647
	Rank 5: Doc 466
	Rank 6: Doc 121
	Rank 7: Doc 586
	Rank 8: Doc 923
	Rank 9: Doc 540
	Rank 10: Doc 1374
Query 6
	Rank 1: Doc 1044
	Rank 2: Doc 544
	Rank 3: Doc 1262
	Rank 4: Doc 311
	Rank 5: Doc 541
